In [1]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata

#EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\base_seguro_de_vida.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [2]:


row = df['e_mail'].isnull() & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'

df.head()


,cooperativa,unidade,cliente,conjunto,coverage,beneficio,placa,motorista,e_mail
0,Stcoop,UNIDADE CASCAVEL,MARCELINO DEBASTIANI,8174,21781,ANUIDADE,<NA>,<NA>,unidade.cascavel@transdesk.com.br
1,Stcoop,UNIDADE RIO VERDE,JEAN CARLOS MACHADO LEAO,8427,21795,SEGURO DE VIDA,<NA>,ALDAIR VITAL DE LIMA,unidade.rio verde@transdesk.com.br
2,Stcoop,UNIDADE RIO VERDE,JEAN CARLOS MACHADO LEAO,8427,21794,ASSISTÊNCIA 24 HORAS,EFO9A01,<NA>,unidade.rio verde@transdesk.com.br
3,Stcoop,UNIDADE RIO VERDE,JEAN CARLOS MACHADO LEAO,8427,21793,VIDROS,EFO9A01,<NA>,unidade.rio verde@transdesk.com.br
4,Stcoop,UNIDADE RIO VERDE,JEAN CARLOS MACHADO LEAO,8427,21792,REPARAÇÃO A TERCEIROS,EFO9A01,<NA>,unidade.rio verde@transdesk.com.br


In [3]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata

def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


In [4]:
df.head()

,cooperativa,unidade,cliente,conjunto,coverage,beneficio,placa,motorista,e_mail
0,Stcoop,UNIDADE CASCAVEL,MARCELINO DEBASTIANI,8174,21781,ANUIDADE,<NA>,<NA>,unidade.cascavel@transdesk.com.br
1,Stcoop,UNIDADE RIO VERDE,JEAN CARLOS MACHADO LEAO,8427,21795,SEGURO DE VIDA,<NA>,ALDAIR VITAL DE LIMA,unidade.rioverde@transdesk.com.br
2,Stcoop,UNIDADE RIO VERDE,JEAN CARLOS MACHADO LEAO,8427,21794,ASSISTÊNCIA 24 HORAS,EFO9A01,<NA>,unidade.rioverde@transdesk.com.br
3,Stcoop,UNIDADE RIO VERDE,JEAN CARLOS MACHADO LEAO,8427,21793,VIDROS,EFO9A01,<NA>,unidade.rioverde@transdesk.com.br
4,Stcoop,UNIDADE RIO VERDE,JEAN CARLOS MACHADO LEAO,8427,21792,REPARAÇÃO A TERCEIROS,EFO9A01,<NA>,unidade.rioverde@transdesk.com.br


Conferência original

In [22]:
df_aggregated = df.groupby(['unidade','beneficio']).agg(
    placas = ('placa','count'),
    motoristas = ('motorista','count')
).reset_index()


df_aggregated[(df_aggregated['placas']>df_aggregated['motoristas']) & (df_aggregated['beneficio']=='SEGURO DE VIDA')]


,unidade,beneficio,placas,motoristas


Carregando para as pastas

In [ ]:
#LOAD

path_to_save = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista'
file_name = 'Cobertura de Seguros Motoristas.xlsx'
final_path = os.path.join(path_to_save, file_name)
os.remove(final_path)
df.to_excel(final_path, engine='openpyxl', index = False)